## Filter
One of the most powerful parameters when defining a search is the ability to filter the results.  While a Query is more of an unstructured approach to search for text within documents, a Filter provides a well-defined structured approach when we know which properties we want to restrict our matching documents.

The following examples will be using some of the popular properties as outlined within the "Properties & Metadata" notebook.  In addition, the selection of these properties was determined based on the discovery techniques used such as metadata and debugging.

In [ ]:
import refinitiv.data as rd
from refinitiv.data.content import search
import pandas as pd
import datetime as dt
import dateutil.relativedelta

# Default session - desktop
rd.open_session()

In [ ]:
# Uncomment the following to see a larger data set when displaying to the screen.
pd.set_option('display.max_colwidth', 140)
#pd.set_option('display.max_rows', 10000)

rd.__version__

#### Example - Query vs Filter
As you become more familiar with Search, you will realize there are a number of different paths you can take to arrive at the same destination.

In [ ]:
# Retrieve the document(s) for a specific ISIN (Astra Zeneca) using a Query.
response=search.Definition(
    view = search.SearchViews.INSTRUMENTS,
    query = 'US046353AQ14',
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
).get_data()
response.data.df

In [ ]:
# Retrieve the document(s) for the same ISIN using a Filter
response=search.Definition(
    view = search.SearchViews.INSTRUMENTS,
    filter = "ISIN eq 'US046353AQ14'",
    select = 'DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID'
).get_data()
response.data.df

#### Example - Apple Equities

In [ ]:
# Within the view (EquityQuotes), select all documents where the term 'AAPL' is within the ticker symbol.  For each, 
# ensure the type of asset is an 'equity' and that each asset is in an active state.  I chose to filter out the 
# AssetState of 'DC' (De-activated) as my criteria. By default, the top 10 results are presented.
#
# Note: An alternative to "AssetType eq 'equity'" could be to use: "RCSAssetCategoryLeaf eq 'Ordinary Share'"
response=search.Definition(
    view = search.SearchViews.EQUITY_QUOTES,
    filter = "AssetType eq 'equity' and AssetState ne 'DC' and TickerSymbol eq 'AAPL'",
    select = "RIC, CompositeDescriptiveName, ExchangeCode, ExchangeName, AssetType"
).get_data()
response.data.df

In [ ]:
response.total

#### Example - Vessels
List all Very Large Gas Containers (VLGC) that are in service within the last week.

In [ ]:
# Build date range expression for the location of the vessel for the last 7 days
date_7_days_ago = dt.datetime.today() - dateutil.relativedelta.relativedelta(days=7)
date_range = f'LocationTimestamp ge {date_7_days_ago.strftime("%Y-%m-%d")} and \
               LocationTimestamp lt {dt.datetime.today().strftime("%Y-%m-%d")}'

In [ ]:
# Show the location of each vessel and some general details such as location, destination and coordinates.
# If available, show the top 10 hits.
response=search.Definition(
    view = search.SearchViews.VESSEL_PHYSICAL_ASSETS,
    filter = f"RCSAssetTypeLeaf eq 'VLGC' and VesselStatus eq 'IN SERVICE' and {date_range}",
    select = 'RIC, SynonymSearchAll, DTSource, LocationTimestamp, IMO,Latitude, Longitude, LocationDestination'
).get_data()
response.data.df

#### Example - Floating Rate Notes
Retrieve all floating rate notes (floating coupon) in the healthcare sector which are issued in Euros.  For each, ensure the coupon rate is great than 2 and our Moody's rating is greater than 'Baa1'.

To determine the rating, we apply the following logic utilizing the RatingsScope property - which is a nested property providing additional details:

- RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10)

The RatingType of 'MIS' refers to the "Moody's Long-term Issuer Rating".
The CurrentRatingRank < 10 instructs search to look for ratings less than 10 ('Baa1').  **Note**: The lower the rank, the better the rating.

In [ ]:
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "CouponRate gt 2 and \
              RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10) and \
              RCSCouponTypeName eq 'Floating Coupon' and \
              IsActive eq true and \
              RCSTRBC2012Name eq 'Healthcare' and \
              RCSCurrencyLeaf xeq 'Euro'",
    select = "CouponCurrency, IssuerName, IssueDate, RIC, CouponRate, ISIN, Ticker, RCSTRBC2012Leaf, MaturityDate, AssetState",   
).get_data()
response.data.df

#### Example - Outstanding Green Bonds
List the outstanding Green Bonds for Telefonica.

In [ ]:
response=search.Definition(
    view = search.SearchViews.GOV_CORP_INSTRUMENTS,
    filter = "IssuerName eq 'Telefonica' and IsGreenBond eq true",
    select = "IssuerName, DocumentTitle, RIC, CouponRate, ISIN, IssuerTicker, DebtTypeDescription, EOMAmountOutstanding, FaceOutstanding"
).get_data()
response.data.df

#### Example - Top executives

In [ ]:
# Using the People View, we can utilize the OfficerDirector property, which contains a number of nested or sub-properties 
# that allow me to perform a more fine-grained search against active people that have been assigned a title code of 
# CEO (Chief Executive Officer). The results are listed based on a default rank defined by the search service.
response=search.Definition(
    view = search.SearchViews.PEOPLE,
    filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')",
    top = 20,
    select = "FullName, PrimaryTitleAsReported, PrimaryOfficerDirector, RIC, OfficerDirectorActiveCompanies, PrimaryOfficerDirectorRIC"
).get_data()
response.data.df

In [ ]:
rd.close_session()